In [1]:
import tensorflow as tf
import numpy as np

**Start an interactive session**

In [2]:
tf.InteractiveSession()

**Create tensors**

In [3]:
x1 = tf.constant(3.0, dtype=tf.float32)
x2 = tf.constant(np.arange(3.0), dtype=tf.float32)
x3 = tf.Variable(tf.random_normal([5, 4, 3]), name="x")

print(x1, x1.eval())
print(x2, x2.eval())
print(x3)

Tensor("Const:0", shape=(), dtype=float32) 3.0
Tensor("Const_1:0", shape=(3,), dtype=float32) [ 0.  1.  2.]
Tensor("x/read:0", shape=(5, 4, 3), dtype=float32)


** Exercise: **
1. What "eval" does?
2. What if I do not create the interactive session, can I still evaluate tensors?
3. Try to run "print x3.eval()", why x3 cannot be evaluated at this stage?

**Perform calculations**

In [4]:
a1 = tf.add(x1,x2)
a2 = tf.multiply(x1,x2)
a3 = tf.subtract(x1,x2)
a4 = tf.divide(x2,x1)

print(a1.eval())
print(a2.eval())
print(a3.eval())
print(a4.eval())

[ 3.  4.  5.]
[ 0.  3.  6.]
[ 3.  2.  1.]
[ 0.          0.33333334  0.66666669]


**More math functions**

In [5]:
a5 = tf.square(x2)
a6 = tf.pow(x2, 2)
a7 = tf.exp(x2)
a8 = tf.cos(x2)

print(a5.eval())
print(a6.eval())
print(a7.eval())
print(a8.eval())

[ 0.  1.  4.]
[ 0.  1.  4.]
[ 1.          2.71828175  7.38905621]
[ 1.          0.54030228 -0.41614684]


**Deal with matrices**

In [6]:
x1 = tf.constant(np.random.randn(4,3), tf.float32) # create a matrix of shape (4,3)
x2 = tf.constant(np.random.randn(3,4), tf.float32) # create a matrix of shape (3,4)

print(x1.get_shape())
print(x2.get_shape())

a1 = tf.transpose(x1) # transpose the matrix
a2 = tf.matmul(x1, x2) # perform the multiplication

print(a1.get_shape(), a1.eval())
print(a2.get_shape(), a2.eval())

(4, 3)
(3, 4)
(3, 4) [[-0.18618293  1.1549269   2.58715439 -0.55028504]
 [ 0.02782409  0.23713699  0.20510338 -0.41397518]
 [-0.20827243 -0.24975823  0.847987   -0.53563541]]
(4, 4) [[ 0.37944287  0.3089734  -0.04486424  0.48277754]
 [-1.05541635 -0.88347697 -0.37227154 -1.56395483]
 [-3.61655402 -2.9084084  -0.2199347  -4.87001753]
 [ 1.71806717  0.72240251 -0.29762328  1.9507823 ]]


For more math operations, please refer to https://www.tensorflow.org/api_guides/python/math_ops

**Let's write linear regression in TensorFlow with placeholder**

In [7]:
# Generate some random data
m = 0.5
b = 2
X = np.arange(0,10.5,0.5) # (21,)
X = X + np.random.randn(len(X), 1).T # (1,21)
Y = m * X + b + (np.random.randn(X.shape[1],1)).T # (1,21)

n_samples = X.shape[1]
learning_rate = 0.001

In [8]:
input_x = tf.placeholder(tf.float32, [1, n_samples]) # define the placeholder for input x
input_y = tf.placeholder(tf.float32, [1, n_samples]) # define the placeholder for input y
# parameters, initialized randomly
m = tf.Variable(np.random.random(), name='slope') # create the variable for slope
b = tf.Variable(np.random.random()*5 - 2.5, name='intercept') # create the variable for intercept
pred = tf.add(tf.multiply(input_x, m), b) # the predicted value
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples) # calculate the loss

# define GD optimization operation
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) 

In [9]:
# operation to initialize all global variables
init = tf.global_variables_initializer() 

# start a new session
# perform the actual training
training_epochs = 50
with tf.Session() as sess: # start a session
    sess.run(init) # run the initialization operation

    # Fit all training data
    for epoch in range(training_epochs):
        # run the optimization and cost operation
        _,c = sess.run([optimizer, cost], feed_dict={input_x: X, input_y: Y})
        print("iteration", epoch, ";", "cost", c)

iteration 0 ; cost 24.1632
iteration 1 ; cost 22.5957
iteration 2 ; cost 21.1441
iteration 3 ; cost 19.7998
iteration 4 ; cost 18.5549
iteration 5 ; cost 17.4021
iteration 6 ; cost 16.3344
iteration 7 ; cost 15.3456
iteration 8 ; cost 14.43
iteration 9 ; cost 13.5819
iteration 10 ; cost 12.7966
iteration 11 ; cost 12.0692
iteration 12 ; cost 11.3956
iteration 13 ; cost 10.7717
iteration 14 ; cost 10.1938
iteration 15 ; cost 9.65866
iteration 16 ; cost 9.16299
iteration 17 ; cost 8.70389
iteration 18 ; cost 8.27866
iteration 19 ; cost 7.8848
iteration 20 ; cost 7.51997
iteration 21 ; cost 7.18204
iteration 22 ; cost 6.86901
iteration 23 ; cost 6.57904
iteration 24 ; cost 6.31043
iteration 25 ; cost 6.06158
iteration 26 ; cost 5.83105
iteration 27 ; cost 5.61747
iteration 28 ; cost 5.41959
iteration 29 ; cost 5.23625
iteration 30 ; cost 5.06637
iteration 31 ; cost 4.90895
iteration 32 ; cost 4.76308
iteration 33 ; cost 4.6279
iteration 34 ; cost 4.50261
iteration 35 ; cost 4.3865
iterati

**Some basic convolutional neural network operations**

In [10]:
# Start with loading MNIST Dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/mnist", one_hot=True)
n_input = 28*28                    # MNIST image feature size

# Let's pick the first several data points for illustration
batch_size = 64
batch_x, batch_y = mnist.train.next_batch(batch_size) # get 64 data points

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/mnist\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/mnist\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/mnist\t10k-labels-idx1-ubyte.gz


In [11]:
# Define placeholders for the input
# tensor shape as (?, 784), set the first dimension to be unknown, 
# as in actual training, batch size is sometimes unknown until runtime
x = tf.placeholder(tf.float32, [None, n_input]) 
x_reshape = tf.reshape(x, shape=[-1, 28, 28, 1]) # reshape to (?, 28, 28, 1)

# Define convolution kernel of shape (5,5,1,32)
cw = tf.Variable(tf.random_normal([5,5,1,32]), name='convolution_weight')

# a convolution operation
conv = tf.nn.conv2d(x_reshape, cw, strides=[1, 2, 2, 1], padding='SAME')
# a pooling operation
pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess: # start a session
    sess.run(init)
    conv_result, pool_result = sess.run([conv, pool], feed_dict={x: batch_x})
    print(conv.get_shape(), pool.get_shape())
    print(conv_result.shape, pool_result.shape)

(?, 14, 14, 32) (?, 7, 7, 32)
(64, 14, 14, 32) (64, 7, 7, 32)
